In [2]:
import os, sys, glob, logging
import inspect, urllib

import pandas, scipy, numpy

import time, datetime, pprint


In [4]:
logging.basicConfig(level=logging.INFO)
# logging.info('logging info')
# logging.warning('logging.warning')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell

import inspect
print(inspect.getfile.__doc__)
print()
src_fn = inspect.getfile(lambda: None)
print(os.path.abspath(src_fn))

Work out which source or compiled file an object was defined in.

/mnt/c/git_work/Xpress/<ipython-input-4-e8f0a355878d>


In [5]:

logging.info("logging")

# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell
__file__ = os.path.abspath(inspect.getfile(lambda: None))

# GTEx data from https://gtexportal.org/home/datasets
# Gene transcripts per million data
download_dir = '/home/zeno/c_drive/Users/Zeno/Downloads'
cache_dir = os.path.join(os.path.dirname(__file__), 'data_cache')
if not os.path.exists(cache_dir):
    logging.info('Createing {}'.format(cache_dir))
    os.makedirs(cache_dir, mode=0o777)

# Detailed data file:
# This file is quite large ~ 1GB
gtex_tpm_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz'
gtex_tpm_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz')
if not os.path.exists(gtex_tpm_fn):
    print("downloading {}".format(gtex_tpm_link))
    #urllib.request.urlretrieve(gtex_tpm_link, gtex_tpm_fn)
# Just looking at median values by tissue (SMTSD) greatly reduces file size
gtex_tpm_med_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz'
gtex_tpm_med_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz')
if not os.path.exists(gtex_tpm_med_fn):
    print("downloading {}".format(gtex_tpm_med_link))
    urllib.request.urlretrieve(gtex_tpm_med_link, gtex_tpm_med_fn)
gtpm_med = pandas.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)

# Sample annotation data
# Main sample data of interest is:
# 'SMTS': 'Tissue Type, area from which the tissue sample was taken.  This is a parent value to SMTSD.'
# 'SMTSD': 'SMTS Detailed'
gtex_attr_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt'
gtex_attr_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SampleAttributesDS.txt')
gtex_attr_desc_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx'
gtex_attr_desc_fn = os.path.join(cache_dir, 'GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx')
if not os.path.exists(gtex_attr_fn):
    print('downloading {}'.format(gtex_attr_link))
    urllib.request.urlretrieve(gtex_attr_link, gtex_attr_fn)
if not os.path.exists(gtex_attr_desc_fn):
    print('downloading {}'.format(gtex_attr_desc_link))
    urllib.request.urlretrieve(gtex_attr_desc_link, gtex_attr_desc_fn)

    
gs = pandas.read_csv(gtex_attr_fn, sep='\t')

tissue_types = sorted(gs['SMTS'].unique())

tissue_dict = dict()
for tissue in tissue_types:
    tissue_dict[tissue] = sorted(gs[gs['SMTS'] == tissue]['SMTSD'].unique())

tissue_subtypes = sorted(gs['SMTSD'].unique())

print(tissue_types)

# Subject Phenotype data
# Age, sex and Hardy Scale death circumstances
gtex_pheno_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SubjectPhenotypesDS.txt'
gtex_pheno_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SubjectPhenotypesDS.txt')


# Just getting a few rows of this giant file
#gtpm_head = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', nrows=1000, usecols=range(3))
GTPM_SKIP = 2
gtpm_ = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', usecols=range(4))

# Other annotation data

# Mutually exclusive 
# Which study was this from?
mut_info_fn = os.path.join(cache_dir, 'enrichments-analysis-result.txt')
#mut_info = pandas.read_csv(mut_info_fn, sep='\t')


# Genes of interest
TEST_GENES = [
    'FAM138A',  # highly expressed
    'TP53'
]

INFO:root:logging


['Adipose Tissue', 'Adrenal Gland', 'Bladder', 'Blood', 'Blood Vessel', 'Bone Marrow', 'Brain', 'Breast', 'Cervix Uteri', 'Colon', 'Esophagus', 'Fallopian Tube', 'Heart', 'Kidney', 'Liver', 'Lung', 'Muscle', 'Nerve', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Salivary Gland', 'Skin', 'Small Intestine', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Uterus', 'Vagina']


In [6]:
def gtpm_row_by_genename(gene_name):
    sub_table = gtpm_[gtpm_['Description'] == gene_name]
    return [GTPM_SKIP + 1 + i for i in sub_table.index][0]

def gtpm_skip_rows(row_ndx, gene_list=TEST_GENES):
    if row_ndx == GTPM_SKIP:
        return False  # header
    if row_ndx in range(GTPM_SKIP):
        return True
    for gn in gene_list:
        if row_ndx in gtpm_row_by_genename(gn):
            return False
    return True

def gtpm_keep_rows(gene_list=TEST_GENES):
    print(gene_list)
    return [GTPM_SKIP] + [gtpm_row_by_genename(gene) for gene in gene_list]

keep_rows = gtpm_keep_rows()
print(keep_rows)

['FAM138A', 'TP53']
[2, 6, 43806]


In [ ]:
print(keep_rows)

start_time = time.time()
gtpm = pandas.read_table(gtex_tpm_fn, skiprows=lambda x: x not in keep_rows, sep='\t', header=0, usecols=range(100))

#gtpm = pandas.read_table(gtex_tpm_fn, skiprows=gtpm_skip_rows, sep='\t',)
end_time = time.time()

print("Tmp table took {} minutes".format((end_time - start_time)/60))


In [9]:
len(tissue_dict)
len(gtpm_med.columns)



55

In [20]:
# Do something with the median values
# pick a single tissue per tissue type
sample_columns = [tissue_dict[t][0] for t in tissue_dict.keys()
                  if tissue_dict[t][0] != 'Cells - Leukemia cell line (CML)']
#print(gtpm_med.head())
gm = gtpm_med.drop('gene_id', axis=1).set_index('Description')
rename_cols = dict([(tissue_dict[t][0], t) for t in tissue_dict.keys()])
gm = gm[sample_columns].rename(columns=rename_cols)
# Copy table so we can add and modify values
gm = gm.copy()
gm.head()
# Using gm table we can chacterize the genes a bit.
# the spread includes extreme values, most of the tissues are similar
# Mean is ~ 16 - 17 TPM, but 50th percentile is 0 and 75th percentile is ~ 2
pandas.set_option('max_columns', 60)
gm.describe()

,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina
Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DDX11L1,0.056945,0.07460,0.05878,0.046275,0.03976,0.089315,0.056480,0.033330,0.05467,0.059490,0.03690,0.06724,0.06347,0.06259,0.06655,0.062785,0.056495,0.03514,0.027255,0.04817,0.065955,0.03569,0.03682,0.06089,0.070250,0.063895,1.76000,0.058350,0.03849,0.03386
WASH7P,11.850000,8.02300,14.24000,8.688500,12.51000,5.743000,12.940000,16.400000,14.20000,13.485000,18.12000,5.36900,10.64000,5.40600,13.68000,6.162500,19.480000,20.53000,5.553000,15.84000,22.400000,11.06000,6.10000,12.19000,18.130000,9.342500,17.14000,19.255000,21.81000,17.28000
MIR1302-11,0.061460,0.08179,0.06097,0.041920,0.04297,0.115450,0.061730,0.030835,0.05660,0.057945,0.08197,0.09328,0.09083,0.08316,0.06216,0.079975,0.067040,0.00000,0.034055,0.04549,0.059580,0.04546,0.04019,0.05562,0.049525,0.078575,0.07375,0.066845,0.00000,0.04732
FAM138A,0.038600,0.04050,0.04113,0.025465,0.02815,0.056265,0.035470,0.041370,0.03914,0.036345,0.04205,0.05190,0.05414,0.03354,0.03790,0.050170,0.034925,0.00000,0.022915,0.02954,0.038015,0.03290,0.03206,0.03527,0.032320,0.043800,0.03714,0.038565,0.00000,0.00000
OR4G4P,0.035695,0.03479,0.00000,0.000000,0.00000,0.050520,0.030855,0.025005,0.03404,0.036230,0.00000,0.03227,0.03431,0.02959,0.00000,0.034130,0.000000,0.00000,0.000000,0.00000,0.028850,0.00000,0.00000,0.03375,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000


In [25]:
gm['mean'] = gm.apply(lambda row: row.mean(), axis=1)
gm['median'] = gm.apply(lambda row: row.median(), axis=1)
gm['std'] = gm.apply(lambda row: row.std(), axis=1)
gm.head()

,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,mean,median,std
Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DDX11L1,0.056945,0.07460,0.05878,0.046275,0.03976,0.089315,0.056480,0.033330,0.05467,0.059490,0.03690,0.06724,0.06347,0.06259,0.06655,0.062785,0.056495,0.03514,0.027255,0.04817,0.065955,0.03569,0.03682,0.06089,0.070250,0.063895,1.76000,0.058350,0.03849,0.03386,0.110681,0.058350,0.301767
WASH7P,11.850000,8.02300,14.24000,8.688500,12.51000,5.743000,12.940000,16.400000,14.20000,13.485000,18.12000,5.36900,10.64000,5.40600,13.68000,6.162500,19.480000,20.53000,5.553000,15.84000,22.400000,11.06000,6.10000,12.19000,18.130000,9.342500,17.14000,19.255000,21.81000,17.28000,13.118917,13.118917,5.084242
MIR1302-11,0.061460,0.08179,0.06097,0.041920,0.04297,0.115450,0.061730,0.030835,0.05660,0.057945,0.08197,0.09328,0.09083,0.08316,0.06216,0.079975,0.067040,0.00000,0.034055,0.04549,0.059580,0.04546,0.04019,0.05562,0.049525,0.078575,0.07375,0.066845,0.00000,0.04732,0.058883,0.059580,0.024157
FAM138A,0.038600,0.04050,0.04113,0.025465,0.02815,0.056265,0.035470,0.041370,0.03914,0.036345,0.04205,0.05190,0.05414,0.03354,0.03790,0.050170,0.034925,0.00000,0.022915,0.02954,0.038015,0.03290,0.03206,0.03527,0.032320,0.043800,0.03714,0.038565,0.00000,0.00000,0.034320,0.036345,0.013486
OR4G4P,0.035695,0.03479,0.00000,0.000000,0.00000,0.050520,0.030855,0.025005,0.03404,0.036230,0.00000,0.03227,0.03431,0.02959,0.00000,0.034130,0.000000,0.00000,0.000000,0.00000,0.028850,0.00000,0.00000,0.03375,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.014668,0.000000,0.017105


In [29]:
print(len(gm))
# high expression # low deviation among tissues
gm_high = gm[(gm['median'] >= 1)  # high expression
             & (gm['median'] - 4 *gm['std'] > 0)
            ]

print(len(gm_high))
gm_high.head()

56202
101


,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,mean,median,std
Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MRPL20,70.695,66.14,72.74,94.785,83.84,47.690,75.685,73.540,87.92,81.295,79.47,53.440,49.19,41.43,71.72,73.765,83.25,72.90,35.520,86.81,87.425,59.93,82.39,56.56,73.240,57.875,56.49,90.250,78.63,67.28,70.396500,72.90,14.677499
SSU72,116.950,84.02,130.70,136.350,112.10,85.165,135.100,147.850,135.10,119.650,142.90,79.230,90.46,63.32,147.50,147.300,116.25,138.60,63.855,156.40,134.650,116.80,151.00,113.10,151.700,100.400,111.30,188.400,142.70,139.60,123.281667,130.70,28.290203
PPCS,45.210,50.79,46.30,48.350,39.61,15.470,43.470,41.915,36.89,35.120,42.05,18.490,36.60,23.46,37.08,15.520,42.75,35.91,22.660,30.59,47.380,42.45,46.17,41.04,46.915,39.660,18.68,49.305,40.95,42.13,37.430500,40.95,10.136836
BOLA1,15.575,19.08,20.87,17.430,19.09,10.705,17.220,18.730,24.83,23.870,22.17,9.501,14.08,10.86,13.04,9.982,22.41,18.70,8.335,16.91,19.265,14.08,18.58,13.79,17.200,15.485,15.98,21.615,21.82,16.58,16.926100,17.20,4.223139
C1orf43,134.050,124.80,132.20,175.850,133.10,94.760,126.300,115.250,114.10,110.400,122.80,93.500,95.85,79.20,142.80,187.100,149.05,104.60,62.900,105.40,113.300,94.29,166.40,90.27,95.640,98.090,95.87,117.950,117.90,106.90,116.687333,114.10,26.807056


In [31]:
gm_high.describe()

,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,mean,median,std
count,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,57.482084,61.009490,61.054772,70.413728,67.232188,44.460777,58.187767,65.094688,65.553277,62.208163,64.307267,48.227327,49.869198,39.082465,58.160931,55.997941,68.502634,64.845020,28.889570,66.407485,65.541609,51.124396,68.409851,51.391554,61.023248,50.520064,58.558376,71.660262,68.506822,60.646030,58.812300,59.437609,13.860251
std,58.020063,61.888289,57.782149,75.223936,76.139214,53.960162,58.470301,66.911595,69.775224,65.171923,65.520329,64.914681,53.124755,41.817030,58.780595,61.838328,69.461484,69.351918,32.633336,77.577985,70.398163,53.042895,70.917735,53.854307,64.197171,52.147076,60.890007,76.380409,68.532964,63.499728,60.652899,60.174968,14.217148
min,2.008500,2.365500,2.416000,1.808500,3.669000,1.274000,1.900000,2.212500,2.070000,2.237500,2.173000,1.548000,1.633000,1.914000,1.839000,1.990000,2.596000,2.043000,0.929600,2.236000,2.203500,1.695000,2.055000,1.529000,1.587500,2.016500,3.066000,2.360500,2.372000,2.093000,2.061370,2.055000,0.493947
25%,21.090000,20.835000,23.290000,25.270000,21.970000,14.515000,21.515000,24.875000,24.200000,22.895000,24.080000,13.830000,15.460000,13.700000,20.940000,16.510000,24.680000,22.070000,10.215000,21.880000,24.025000,18.990000,25.520000,19.140000,24.185000,16.985000,20.220000,27.570000,25.310000,23.210000,22.780450,23.466050,5.395549
50%,37.940000,44.750000,40.770000,47.835000,40.950000,23.960000,37.510000,40.660000,38.150000,36.960000,42.050000,27.310000,31.050000,25.770000,37.440000,33.560000,43.550000,40.580000,17.585000,41.480000,40.530000,33.920000,42.030000,33.430000,40.780000,34.030000,35.480000,44.550000,42.280000,39.740000,37.308667,39.590000,9.287707
75%,68.815000,69.085000,74.950000,76.940000,78.750000,53.210000,67.915000,81.790000,77.090000,75.035000,79.470000,51.380000,54.080000,42.850000,68.860000,64.510000,83.250000,78.810000,36.115000,83.150000,80.325000,59.930000,84.060000,57.650000,72.690000,57.055000,73.630000,81.855000,78.800000,70.560000,67.735667,68.520000,16.263994
max,276.050000,355.000000,290.700000,420.450000,407.200000,281.600000,277.150000,335.000000,359.700000,343.600000,340.200000,341.100000,265.700000,206.900000,292.700000,312.950000,370.550000,389.100000,222.900000,471.400000,395.650000,290.200000,381.500000,291.700000,387.650000,245.400000,380.100000,420.350000,361.800000,309.200000,299.504500,300.600000,75.012766


In [ ]:

gt = gtpm[gtpm.columns[1:]]
gt.set_index('Description', inplace=True)
gtt = gt.transpose()
gtt.head()

In [ ]:
gtt.head().iplot(kind='hist', yTitle='TPM')

In [ ]:
print(gt.head())
gt.iplot(kind='hist', yTitle='TPM')


In [ ]:
gtt['TP53'].iplot(kind='hist', yTitle='TPM')